In [1]:

from neo4j import GraphDatabase

# URI examples: "neo4j://localhost", "neo4j+s://xxx.databases.neo4j.io"
URI = "neo4j://localhost"
AUTH = ("neo4j", "GSW30_curry")



In [2]:
with GraphDatabase.driver(URI, auth=AUTH) as driver:
    driver.verify_connectivity()
    driver.execute_query(
        "CREATE CONSTRAINT id_unidade_conservacao_constraint FOR (r:UnidadeConservacao) REQUIRE r.id_unidade_conservacao IS UNIQUE"
    )
    driver.execute_query(
        "CREATE CONSTRAINT nome_unidade_conservacao_constraint FOR (r:UnidadeConservacao) REQUIRE r.nome IS UNIQUE"
    )

In [3]:
with GraphDatabase.driver(URI, auth=AUTH) as driver:
    driver.verify_connectivity()
    driver.execute_query(
        "LOAD CSV WITH HEADERS FROM 'http://localhost:11001/project-013a6b60-6f90-4782-9618-c6b239c477da/unidade_conservacao.csv' AS csvLine "
        "CREATE (r:UnidadeConservacao {id_unidade_conservacao: toInteger(csvLine.id_unidade_conservacao), nome: csvLine.nome, categoria: csvLine.categoria, "
        "esfera_administrativa: csvLine.esfera_administrativa, tipo: csvLine.tipo, area_amazonia_ha: toFloat(csvLine.area_amazonia_ha)})"
    )

In [4]:
with GraphDatabase.driver(URI, auth=AUTH) as driver:
    driver.verify_connectivity()
    driver.execute_query(
        "LOAD CSV WITH HEADERS FROM 'http://localhost:11001/project-013a6b60-6f90-4782-9618-c6b239c477da/unidade_conservacao.csv' AS csvLine "
        "MATCH (e:Estado {id_estado: toInteger(csvLine.id_estado)}) "
        "MATCH (uc:UnidadeConservacao {id_unidade_conservacao: toInteger(csvLine.id_unidade_conservacao)}) "
        "MERGE (uc)-[:Pertence]->(e) "
        )